In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import yfinance as yf
import matplotlib.pyplot as plt
from scipy.stats import norm
from urllib.request import urlopen
import certifi
import json
import math

plt.rcParams["figure.figsize"] = (16,9)

**Forward Price**

In [10]:
def get_currency_fwd(spot_rate_domestic_forien,domestic_rate_pct,forien_rate_pct,t_years,is_cont=False):
    
    '''
    rate is in perecent
    Replication (Domestic: Yen , Forien: USD)
    1. Borrow Forien: USD 
    2. Sell Forien: USD for Domestic: Yen
    3. Invest the in Domestic: Yen rate
    '''
    # continous case
    if is_cont: return spot_rate_domestic_forien*np.exp((domestic_rate_pct-forien_rate_pct)/100)

    return spot_rate_domestic_forien*(1+domestic_rate_pct/100)/(1+forien_rate_pct/100)

def get_fwd_price(spot_rate_domestic_forien,domestic_rate_pct,forien_rate_pct,t_years,is_cont=False):
    
    '''
    rate is in perecent
    Replication (Domestic: Yen , Forien: USD)
    1. Borrow Forien: USD 
    2. Sell Forien: USD for Domestic: Yen
    3. Invest the in Domestic: Yen rate
    '''
    # continous case
    if is_cont: return spot_rate_domestic_forien*np.exp((domestic_rate_pct-forien_rate_pct)/100)

    return spot_rate_domestic_forien*(1+domestic_rate_pct/100)/(1+forien_rate_pct/100)

In [11]:
get_currency_fwd(120,0.15,2.40,1,True)


117.33014846320036